# Assignment 1

## Exercise 4

### Instructions

Consider the FrozenLake8x8-v1 environment. It is similar to the FrozenLake-v1 that was studied
in Tinkering Notebook 2, but it consist of an 8 × 8 grid and thus have 64 states.

Write a code that find an optimal policy π∗(s) and the corresponding value function v∗(s).

In the quizz on you will be asked for example ”Which of these are optimal actions in state s = 26?”
or ”What is v∗(26)?”. So make sure that you can easily run code that can answer these types of
questions for different states.

Hint: You can check that your code seems to be working by ensuring that you get to correct answer
to the following:

- For the optimal policy v∗(26) = 0.80 (rounded to two decimals).
- In s = 26 the optimal action is 0 (left).
